In [1]:
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16
import tensorflow as tf
import os
import numpy as np
import string
import random
import pickle

2022-08-02 19:04:25.081737: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-02 19:04:25.081811: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
IMAGES_PATH = "flickr30k_images/flickr30k_images/"
IMAGE_TARGET_SIZE = (224, 224)

def loadData(pathToFile):
    f = open(pathToFile)
    lines = f.readlines()[1:]
    captionMap = {}
    for line in lines:
        dataLine = line.strip().split('|')
        if len(dataLine) == 3:
            imageName = dataLine[0].strip()
            imageName = os.path.join(imageName)
            imageCaption = dataLine[2].strip()
            
            if imageName.endswith(".jpg"):
                if imageName in captionMap:
                    captionMap[imageName].append(imageCaption)
                else:
                    captionMap[imageName] = list()
                    captionMap[imageName].append(imageCaption)
    return captionMap
        
def trainTestSplit(captionMap, trainPerc):
    if trainPerc < 1 or trainPerc > 99:
        raise Exception("Define an appropriate train split percentage")
    
    imgList = list(captionMap.keys())
    np.random.shuffle(imgList)
    
    trainingSetSize = int(len(imgList) * (trainPerc / 100))
    trainingSet = {
        imageName: captionMap[imageName] for imageName in imgList[:trainingSetSize] 
    }
    testSet = {
        imageName: captionMap[imageName] for imageName in imgList[trainingSetSize:]
    }
    
    return trainingSet, testSet
        

In [3]:
def cleanDescr(captionMap):
    for _, captionList in captionMap.items():
        for i in range(len(captionList)):
            cap = captionList[i]
            cap = [ch for ch in cap if ch not in string.punctuation]
            cap = ''.join(cap)
            cap = cap.split(' ')
            cap = [word.lower() for word in cap if len(word)>1 and word.isalpha()]
            cap = ' '.join(cap)
            captionList[i] = cap

def generateVocabulary(captionMap):
    words = set()
    for key in captionMap.keys():
        for cap in captionMap[key]:
            words.update(cap.split())
    return words

def takeSubsample(captionMap, k):
    newCaption = {}
    for _ in range(k):
        randomKey = random.choice(list(captionMap.keys()))
        captions = captionMap[randomKey]
        captionMap.pop(randomKey)
        
        newCaption[randomKey] = captions
    return newCaption
            

In [4]:
seed = 39
np.random.seed(seed)
random.seed(seed)

captionMap = loadData("flickr30k_images/results.csv")
captionMap = takeSubsample(captionMap, 8000)

cleanDescr(captionMap)
vocab = generateVocabulary(captionMap)

trainingMap, testMap = trainTestSplit(captionMap, 70)

print("Training samples:", len(trainingMap))
print("Test samples:", len(testMap))

Training samples: 5600
Test samples: 2400


In [5]:
def processImage(imageName):
    imagePath = os.path.join(IMAGES_PATH, imageName)
    image = load_img(imagePath, target_size = IMAGE_TARGET_SIZE, interpolation="nearest")
    imgArr = img_to_array(image)
    # Add one more dimension
    imgArr = np.expand_dims(imgArr, axis = 0)
    imgArr = preprocess_input(imgArr)
    return imgArr

def encode(image, model):
    imageCoded = processImage(image)
    vec = model.predict(imageCoded, verbose=1)
    vec = np.reshape(vec, (vec.shape[1]))
    return vec

base_model = VGG16()
model = Model(base_model.input, base_model.layers[-2].output)
features = {}
for img in trainingMap:
    features[img] = encode(img, model)

#store features
pickle.dump(features, open('features.pkl', 'wb'))



2022-08-02 19:04:51.446289: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-02 19:04:51.446369: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-02 19:04:51.446428: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dosclic98-350V5C-351V5C-3540VC-3440VC): /proc/driver/nvidia/version does not exist
2022-08-02 19:04:52.232755: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2022-08-02 19:04:52.837797: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2022-08-02 19:04:52.980765: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2022

1/1 [==============================] - 1s 885ms/step
